<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 2.2.1 *Querying the International Space Station*

# The OpenNotify API

The [OpenNotify API](http://open-notify.org/Open-Notify-API/ISS-Location-Now/) exposes a few attributes of the International Space Station (ISS) via a simple, authentication-free interface. The simplicity of this API precludes any need for a dedicated Python library. However, as with many APIs, it accepts requests according to HTTP standards and returns responses in JSON format, so the Python libraries request and json will make managing the I/O simpler still.

In [1]:
import requests
import json
from datetime import datetime, date, time

## Check ISS location

This request fetches the latest position of the international space station:

In [2]:
response = requests.get("http://api.open-notify.org/iss-now.json")

http://api.open-notify.org/iss-now.json

```json
{
  "message": "success", 
  "timestamp": UNIX_TIME_STAMP, 
  "iss_position": {
    "latitude": CURRENT_LATITUDE, 
    "longitude": CURRENT_LONGITUDE
  }
}
```

### Status code of the response

In [3]:
# Answer
response.status_code

200

### Location of the ISS

In [4]:
# Answer
response.json()

{'iss_position': {'longitude': '-113.7582', 'latitude': '27.1569'},
 'timestamp': 1637060125,
 'message': 'success'}

## Check when ISS will be overhead at your location

We can also request the current position of the ISS and the next few times at which it will be over a certain location. The latitude and longitude of Sydney are (-33.87, 151.21). Create a dict named `parameters` whose elements are key:value pairs named `lat` and `lon`, containing those figures:

http://api.open-notify.org/iss-pass.json?lat=LAT&lon=LON

```json
{
  "message": "success",
  "request": {
    "latitude": LATITUE,
    "longitude": LONGITUDE, 
    "altitude": ALTITUDE,
    "passes": NUMBER_OF_PASSES,
    "datetime": REQUEST_TIMESTAMP
  },
  "response": [
    {"risetime": TIMESTAMP, "duration": DURATION},
    ...
  ]
}
```

In [26]:
parameters = {"lat" : 1.3521, "lon" : 103.8198}

The API request we want starts like the last one, but the last part of the endpoint is "iss-pass.json", and it takes a second argument -- namely, the `parameters` value created above. Compose and execute the API request:

In [34]:
url = f"http://api.open-notify.org/iss-pass.json?lat={parameters['lat']}&lon={parameters['lon']}"

In [35]:
R = requests.get(url)

In [36]:
R.json()

{'message': 'success',
 'request': {'altitude': 100,
  'datetime': 1637065936,
  'latitude': 1.3521,
  'longitude': 103.8198,
  'passes': 5},
 'response': [{'duration': 344, 'risetime': 1637093723},
  {'duration': 648, 'risetime': 1637099375},
  {'duration': 652, 'risetime': 1637141200},
  {'duration': 368, 'risetime': 1637147146},
  {'duration': 644, 'risetime': 1637182957}]}

### Response header

In [37]:
header = requests.head(url)
header

<Response [200]>

In [44]:
R.headers

{'Server': 'nginx/1.10.3', 'Date': 'Tue, 16 Nov 2021 12:32:16 GMT', 'Content-Type': 'application/json', 'Content-Length': '523', 'Connection': 'keep-alive', 'Via': '1.1 vegur'}

In [38]:
response.header("http://api.open-notify.org/iss-now.json")
response

AttributeError: 'Response' object has no attribute 'header'

### Content of the response

Print the content of the response (the data that the server returned):

In [39]:
R.text

'{\n  "message": "success", \n  "request": {\n    "altitude": 100, \n    "datetime": 1637065936, \n    "latitude": 1.3521, \n    "longitude": 103.8198, \n    "passes": 5\n  }, \n  "response": [\n    {\n      "duration": 344, \n      "risetime": 1637093723\n    }, \n    {\n      "duration": 648, \n      "risetime": 1637099375\n    }, \n    {\n      "duration": 652, \n      "risetime": 1637141200\n    }, \n    {\n      "duration": 368, \n      "risetime": 1637147146\n    }, \n    {\n      "duration": 644, \n      "risetime": 1637182957\n    }\n  ]\n}\n'

Note that this is a Python byte string:

In [40]:
R.json()

{'message': 'success',
 'request': {'altitude': 100,
  'datetime': 1637065936,
  'latitude': 1.3521,
  'longitude': 103.8198,
  'passes': 5},
 'response': [{'duration': 344, 'risetime': 1637093723},
  {'duration': 648, 'risetime': 1637099375},
  {'duration': 652, 'risetime': 1637141200},
  {'duration': 368, 'risetime': 1637147146},
  {'duration': 644, 'risetime': 1637182957}]}

Print just the "content-type" value from the header:

In [45]:
R.headers["content-type"]

'application/json'

### Decode JSON byte string

JSON was designed to be easy for computers to read, not for people. The `requests` library can decode the JSON byte string:

In [46]:
R.json()

{'message': 'success',
 'request': {'altitude': 100,
  'datetime': 1637065936,
  'latitude': 1.3521,
  'longitude': 103.8198,
  'passes': 5},
 'response': [{'duration': 344, 'risetime': 1637093723},
  {'duration': 648, 'risetime': 1637099375},
  {'duration': 652, 'risetime': 1637141200},
  {'duration': 368, 'risetime': 1637147146},
  {'duration': 644, 'risetime': 1637182957}]}

What kind of object did this give us?

In [0]:
#Dictionary 

Python dicts are easier to work with, but the data we want is still buried in that data structure, so we have to dig it out. First, extract the `response` value to a separate dict:

In [49]:
R_response = R.json()['response']
R_response

[{'duration': 344, 'risetime': 1637093723},
 {'duration': 648, 'risetime': 1637099375},
 {'duration': 652, 'risetime': 1637141200},
 {'duration': 368, 'risetime': 1637147146},
 {'duration': 644, 'risetime': 1637182957}]

Now extract the `risetime` strings into an array called `srisetimes`:

In [81]:
risetimes = R_response[0]["risetime"] 
risetimes

1637093723

In [84]:
#[r["risetime"] for r in R_response]
srisetimes = [h["risetime"] for h in R_response]
srisetimes

[1637093723, 1637099375, 1637141200, 1637147146, 1637182957]

```
datetime.fromtimestamp(str)
```

Convert these to an array of Python `datetime` values called `risetimes`:

Finally, print these in a format that people understand:



```
str.strftime('%d/%m/%y %I:%M')

27/03/19 10:59
27/03/19 12:35
27/03/19 02:11
28/03/19 03:42
28/03/19 05:13
```



## Check who is onboard the ISS

In [0]:
response = requests.get("http://api.open-notify.org/astros.json")

Referring to the methods used above, extract the number of astronauts and their names:

# HOMEWORK


1. Write a simple handler for the response status code (refer to lab resources slide for HTTP response codes). As this Jupyter Notebook is an interactive device, the handler does not need to manage subsequent code execution (i.e. by branching or aborting execution), although it should return something that could be used to do so if deployed in a Python program.

In [0]:
#ANSWER:
def handleResponse(response, verbose = False):
    '''
    Returns Boolean Value, Status Code, 
    '''
  # if Status Code is 200 return false, and status code
  # Otherwise Return True and Status Code

2. Test your response handler on some correct and incorrect API calls.

In [0]:
response = requests.get("http://api.open-notify.org/astros.json")
if handleResponse(response)[0]:
    print('API call failed. Resolve issue before continuing!')
    
response = requests.get("http://api.open-notify.org/iss-pass.json")
handleResponse(response, True)[0]